In [1]:
#import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

learning_rate = 0.0002
gamma         = 0.98

ModuleNotFoundError: No module named 'torch'

In [2]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []
        
        self.fc1 = nn.Linear(100, 128)
        self.fc2 = nn.Linear(128, 100)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x
      
    def put_data(self, item):
        self.data.append(item)
        
    def train_net(self):
        R = 0
        for r, prob in self.data[::-1]:
            R = r + gamma * R
            loss = -torch.log(prob) * R
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        self.data = []

In [8]:
def main():
    env = gym.make('CartPole-v1')
    pi = Policy()
    score = 0.0
    print_interval = 20
    
    for n_epi in range(10000):
        s = env.reset()
        
        for t in range(501): # CartPole-v1 forced to terminates at 500 step.
            env.render()
            prob = pi(torch.from_numpy(s).float())
            m = Categorical(prob)
            a = m.sample()
            s_prime, r, done, info = env.step(a.item())
            pi.put_data((r,prob[a]))
            
            s = s_prime
            score += r
            if done:
                break

        pi.train_net()
        
        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
            score = 0.0
    env.close()

In [9]:
env = gym.make('CartPole-v1')
pi = Policy()
score = 0.0
print_interval = 20
s = env.reset()
print(s)
#env.render()
prob = pi(torch.from_numpy(s).float())
print(prob)
m = Categorical(prob)
print(m)
a = m.sample()
print(a)
s_prime, r, done, info = env.step(a.item())
pi.put_data((r,prob[a]))
            
s = s_prime
score += r

[ 0.03419508  0.02373214 -0.04085779 -0.04349106]
tensor([0.5520, 0.4480], grad_fn=<SoftmaxBackward>)
Categorical(probs: torch.Size([2]))
tensor(1)


In [ ]:
if __name__ == '__main__':
    main()

# of episode :20, avg score : 27.5
# of episode :40, avg score : 25.45
# of episode :60, avg score : 34.6
# of episode :80, avg score : 40.3
# of episode :100, avg score : 40.25
# of episode :120, avg score : 52.0
# of episode :140, avg score : 41.75
# of episode :160, avg score : 64.95
# of episode :180, avg score : 61.8
# of episode :200, avg score : 93.9
# of episode :220, avg score : 155.2
# of episode :240, avg score : 134.95
# of episode :260, avg score : 162.55
# of episode :280, avg score : 184.5
# of episode :300, avg score : 185.85
# of episode :320, avg score : 205.85
# of episode :340, avg score : 152.75
# of episode :360, avg score : 180.9
# of episode :380, avg score : 291.2
# of episode :400, avg score : 180.4
# of episode :420, avg score : 298.5
# of episode :440, avg score : 166.2
# of episode :460, avg score : 133.6
# of episode :480, avg score : 136.05
# of episode :500, avg score : 140.55
# of episode :520, avg score : 120.05
# of episode :540, avg score : 169.7
# o